In [1]:
import pandas as pd
import numpy as np
import math
import scipy.optimize as opt
from geopy.geocoders import GoogleV3
import googlemaps
import folium
import polyline

In [3]:
API_KEY = ''
geolocator = GoogleV3(api_key=API_KEY)
gmaps = googlemaps.Client(key=API_KEY)

url = 'https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population'
tables = pd.read_html(url)
df = tables[0]
df.head()

,Neighborhood,Population (2010),NPU
0,Adair Park,1331,V
1,Adams Park,1763,R
2,Adamsville,2403,H
3,Almond Park,1020,G
4,Ansley Park,2277,E


In [9]:
df.columns = ['Neighborhood', 'Population', 'NPU']
df = df.dropna()

top10_neighborhoods = df.nlargest(10, 'Population')
top10_neighborhoods

,Neighborhood,Population,NPU
95,Midtown,16569,E
51,Downtown,13411,M
104,Old Fourth Ward,10505,M
101,North Buckhead,8270,B
119,Pine Hills,8033,B
98,Morningside/Lenox Park,8030,F
149,Virginia-Highland,7800,F
66,Grant Park,6771,W
64,Georgia Tech,6607,E
80,Kirkwood,5897,O


In [101]:
def get_geocode(location):
    coords = geolocator.geocode(location + ', Atlanta, GA')
    return (coords.latitude, coords.longitude) if coords else (np.nan, np.nan)

top10_neighborhoods[['Lat', 'Lng']] = top10_neighborhoods['Neighborhood'].apply(lambda x: pd.Series(get_geocode(x)))
top10_neighborhoods = top10_neighborhoods.dropna()

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi, delta_lambda = np.radians(lat2 - lat1), np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

def total_cost_haversine(coords, data):
    lat, lon = coords
    distances = data.apply(lambda row: haversine(lat, lon, row['Lat'], row['Lng']), axis=1)
    return (data['Population'] * distances).sum()

initial_guess = [top10_neighborhoods['Lat'].mean(), top10_neighborhoods['Lng'].mean()]
bounds = [(top10_neighborhoods['Lat'].min(), top10_neighborhoods['Lat'].max()),
          (top10_neighborhoods['Lng'].min(), top10_neighborhoods['Lng'].max())]

result_haversine = opt.minimize(total_cost_haversine, initial_guess, args=(top10_neighborhoods,), bounds=bounds)
optimal_location_haversine = result_haversine.x
optimal_address_haversine = geolocator.reverse(optimal_location_haversine).address
cost_haversine = result_haversine.fun

In [103]:
def driving_distance(origin, destination):
    directions = gmaps.directions(origin, destination, mode='driving')
    return directions[0]['legs'][0]['distance']['value'] / 1000 if directions else np.nan

def total_cost_driving(coords, data):
    lat, lon = coords
    distances = data.apply(lambda row: driving_distance((lat, lon), (row['Lat'], row['Lng'])), axis=1)
    return (data['Population'] * distances).sum()

result_driving = opt.minimize(total_cost_driving, initial_guess, args=(top10_neighborhoods,), method='SLSQP',bounds=bounds)
optimal_location_driving = result_driving.x
optimal_address_driving = geolocator.reverse(optimal_location_driving).address
cost_driving = result_driving.fun

In [105]:
m = folium.Map(location=initial_guess, zoom_start=12)

for _, row in top10_neighborhoods.iterrows():
    folium.Marker([row['Lat'], row['Lng']], popup=row['Neighborhood']).add_to(m)

folium.Marker(optimal_location_haversine, popup=f'Haversine Optimal Location\n{optimal_address_haversine}',
              icon=folium.Icon(color='red')).add_to(m)
folium.Marker(optimal_location_driving, popup=f'Driving Optimal Location\n{optimal_address_driving}',
              icon=folium.Icon(color='green')).add_to(m)

for _, row in top10_neighborhoods.iterrows():
    start, end = optimal_location_driving, (row['Lat'], row['Lng'])
    directions = gmaps.directions(start, end, mode="driving")
    if directions:
        points = polyline.decode(directions[0]['overview_polyline']['points'])
        folium.PolyLine(locations=points, color='blue', weight=2.5, opacity=1).add_to(m)

m.save('C:/Users/19707/Desktop/atlanta_pickup_locations..html')

print(f"Haversine Method - Optimal Location: {optimal_location_haversine}, Address: {optimal_address_haversine}, Cost: {cost_haversine:.2f}")
print(f"Driving Distance Method - Optimal Location: {optimal_location_driving}, Address: {optimal_address_driving}, Cost: {cost_driving:.2f}")

Haversine Method - Optimal Location: [ 33.77914624 -84.37640394], Address: 343 8th St NE, Atlanta, GA 30309, USA, Cost: 298373.08
Driving Distance Method - Optimal Location: [ 33.78371838 -84.36741019], Address: 600 Cresthill Ave NE, Atlanta, GA 30306, USA, Cost: 387630.99
